In [ ]:
import { load } from "dotenv";
const env = await load();

const process = { env };
process.env;


In [ ]:
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";
import { TextLoader } from "langchain/document_loaders/fs/text";
import { OpenAIEmbeddings } from "@langchain/openai";

const loader = new TextLoader("documents/data.txt");
const docs = await loader.load();

const splitter = new RecursiveCharacterTextSplitter({
  chunkSize: 100,
  chunkOverlap: 20,
});

const splitDocs = await splitter.splitDocuments(docs);

const embeddings = new OpenAIEmbeddings({
  model: process.env.EMBEDDING_MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

// embeddings
console.log(embeddings.modelName);

await embeddings.embedQuery(splitDocs[0].pageContent);


In [ ]:
import { ChatMessageHistory } from "langchain/memory";
import { HumanMessage, AIMessage } from "@langchain/core/messages";
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";

const chatHistory = new ChatMessageHistory();

// await chatHistory.addUserMessage('你好')
// await chatHistory.addAIChatMessage('有什么可以帮你的？')

await chatHistory.addMessage(new HumanMessage("hi"));
await chatHistory.addMessage(new AIMessage("What can I do for you?"));

const messages = await chatHistory.getMessages();

// console.log(messages);
const chatModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const prompt = ChatPromptTemplate.fromMessages([
  ["system", "Your are a {systemText}"],
  new MessagesPlaceholder("chat_history"),
  ["human", "{humanText}"],
]);

const chain = prompt.pipe(chatModel);

await prompt.invoke({
  systemText: "Assistant is a large language model",
  humanText: "Hello",
  chat_history: messages,
})


In [ ]:
import { ChatMessageHistory } from "langchain/memory";
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { RunnableWithMessageHistory } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";

const chatHistory = new ChatMessageHistory();

// console.log(messages);
const chatModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const prompt = ChatPromptTemplate.fromMessages([
  ["system", "Your are a chat bot"],
  new MessagesPlaceholder("chat_history"),
  ["human", "{humanText}"],
]);

const outputParse = new StringOutputParser();

const chain = prompt.pipe(chatModel).pipe(outputParse);

const chainWithHistory = new RunnableWithMessageHistory({
  runnable: chain,
  getMessageHistory: () => chatHistory,
  inputMessagesKey: "humanText",
  historyMessagesKey: "chat_history",
});

const res1 = await chainWithHistory.invoke(
  {
    humanText: "Hello, I'm Bob",
  },
  {
    configurable: { sessionId: "none" },
  }
);
console.log(res1);

await chainWithHistory.invoke(
  {
    humanText: "Who am I?",
  },
  {
    configurable: { sessionId: "none" },
  }
);
